In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from os import listdir

In [34]:
"""
    Camera calibration 
    
    Compute #mm/ pixel object displacement

    Calculate horizontal field of view(FOV) in degrees of angle.

    Calculate theoretical values and compare with measured values.

    Use lens of focal length: 18mm, 53mm, 135mm

    Object displacement of: 1mm, 5mm, 10mm, 20mm

    Object distance of: 0.6m, 1.2m, 1.8m
    
    3 * 4 * 3 = 36 values
    
    #Sensor size: 23.4 x 15.6 mm
"""

pi = 3.14159265358979

def FOV_theoretical(f, sensor_width=23.4):
    return 2 * np.arctan( sensor_width/(2*f) ) * 180 / pi

def FOV_measured(image_width, mm_per_pixel, object_distance):
    return 2 * np.arctan( image_width * mm_per_pixel / (2 * object_distance) ) * 180 / pi

In [36]:
"""
    Images: imgA, imgB
    Detect motions vectors between imgA and imgB.
    31×31 block size.
    Threshold of search range: 400 pixels. 
"""

def pixelShift(imgA, imgB, bSize=31, rangeLen=200):
    #0 1 2 3 4
    imgLen = min(imgA.shape[0], imgA.shape[1])
    bLen = min(imgA.shape[0], imgA.shape[1]) // bSize
    #print(bLen)
    ArrU = []
    ArrV = []
    for i in range(bLen): #
        for j in range(bLen): # for all imgA sliding windows(block) -> compare to all imgB block
            minValue = 1e+9
            curMinValue = 1e+9
            tempA, tempB = 0, 0
            leftIndex = i*bSize
            rightIndex = j*bSize
            for a in range(-50, 50):
                if i*bSize + a < 0 or i*bSize + a + bSize >= imgLen:
                    continue
                for b in range(rangeLen):
                    if j*bSize + b < 0 or j*bSize + b + bSize >= imgLen:
                        continue
                    #https://stackoverflow.com/questions/26445153/iterations-through-pixels-in-an-image-are-terribly-slow-with-python-opencv
                    #print(leftIndex, leftIndex+bSize, rightIndex, rightIndex+bSize, leftIndex + a, leftIndex + a + bSize, rightIndex + b, rightIndex + b + bSize)
                    curMinValue = np.sum(np.abs(imgA[leftIndex:leftIndex+bSize, rightIndex:rightIndex+bSize] - imgB[leftIndex + a:leftIndex + a + bSize, rightIndex + b:rightIndex + b + bSize]))
                    if curMinValue < minValue:
                        minValue = curMinValue
                        tempA = a
                        tempB = b
            ArrU.append(tempA)
            ArrV.append(tempB)
    ArrU = np.array(ArrU)
    ArrV = np.array(ArrV)
    print(ArrV)
    return np.max(ArrV)


In [ ]:
"""
    lens of focal length: 18mm
    Object displacement of: 1mm, 5mm, 10mm, 20mm
    Object distance of: 0.6m, 1.2m, 1.8m
"""
filePath = "./photo/18mm"

imgs600 = []
configs600 = []

imgs1200 = []
configs1200 = []

imgs1800 = []
configs1800 = []
print(listdir(filePath))
for file in listdir(filePath):
    if file[-3:] == "jpg" or file[-3:] == "JPG":
        img = cv2.imread(filePath + '/' + file, 0)
        img = img[1200:2000, 2000:2800]
        config = file[:-4].split('_')
        config = [int(config[0][:-2]), int(config[1][:-2])]
        print(img.shape)
        print(config)
        plt.imshow(img, cmap="gray")
        plt.show()
        if config[0] == 600:
            imgs600.append(img)
            configs600.append(config)
        elif config[0] == 1200:
            imgs1200.append(img)
            configs1200.append(config)
        else:
            imgs1800.append(img)
            configs1800.append(config)

In [ ]:
#0 1 5 10 20

for i in range(1, 5):
    print(pixelShift(imgs600[0], imgs600[i]), configs600[i])

In [42]:
print(FOV_theoretical(f=18))
#600
#(2280, 1916)

#(2286, 1916) +6
#(2307, 1914) +27
#(2334, 1913) +54
#(2382, 1912) +102
mmPerPixel = 1/6
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 5/27
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 10/54
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 20/102
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 


#1200
#(2337, 1747)

#(2339, 1747) +2
#(2353, 1746) +14
#(2366, 1746) +29
#(2393, 1746) +56
mmPerPixel = 1/2
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 5/14
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 10/29
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 20/56
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 

#1800
#(2325, 1685)

#(2328, 1684) +3
#(2334, 1684) +9
#(2344, 1684) +19
#(2370, 1684) +45
mmPerPixel = 1/3
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 5/9
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 10/19
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 20/45
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800))

66.04773511159335
0.16666666666666666 65.95818248913486
0.18518518518518517 71.58227651971828
0.18518518518518517 71.58227651971828
0.19607843137254902 74.71613366088047
0.5 88.45155967014396
0.35714285714285715 69.61699629994469
0.3448275862068966 67.74410913453946
0.35714285714285715 69.61699629994469
0.3333333333333333 46.78590291091027
0.5555555555555556 71.5822765197183
0.5263157894736842 68.66935080774333
0.4444444444444444 59.95184734379076


In [44]:
print(FOV_theoretical(f=53))
#600
#(2289, 2301)

#(2311, 2300) +22
#(2352, 2299) +63
#(2419, 2296) +130
#(2554, 2287) +265
mmPerPixel = 1/22
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 5/63
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 10/130
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 20/265
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 


#1200
#(2397)

#(2407) +10
#(2441) +44
#(2474) +77
#(2559) +162
mmPerPixel = 1/10
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 5/44
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 10/77
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 20/162
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 

#1800
#2407

#2418 +11
#2444 +37
#2465 +58
#2509 +102
mmPerPixel = 1/11
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 5/37
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 10/58
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 20/102
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800))

24.897315580182365
0.045454545454545456 20.071421184197042
0.07936507936507936 34.34174457156521
0.07692307692307693 33.34456810728294
0.07547169811320754 32.74942406420271
0.1 22.031617758871587
0.11363636363636363 24.94728981789867
0.12987012987012986 28.375761933661114
0.12345679012345678 27.027134864055434
0.09090909090909091 13.457280726997274
0.13513513513513514 19.894225015532207
0.1724137931034483 25.22491581549244
0.19607843137254902 28.55366537772997


In [45]:
print(FOV_theoretical(f=135))
#600
#2259

#2316 +17
#2421 +162
#2525 +266
#2764 +505
mmPerPixel = 1/17
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 5/162
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 10/266
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 
mmPerPixel = 20/505
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 600)) 


#1200
#2505

#2541 +36
#2566 +61
#2648 +143
#2815 +310
mmPerPixel = 1/36
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 5/61
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 10/143
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 
mmPerPixel = 20/310
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1200)) 

#1800
#2326

#2330 +4
#2360 +34
#2415 +89
#2540 +214
mmPerPixel = 1/4
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 5/34
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 10/89
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800)) 
mmPerPixel = 20/214
print(mmPerPixel, FOV_measured(4672, mmPerPixel, 1800))

9.906514955684132
0.058823529411764705 25.798805977808914
0.030864197530864196 13.704141502511817
0.03759398496240601 16.65404382479744
0.039603960396039604 17.530972220889325
0.027777777777777776 6.190403557130998
0.08196721311475409 18.13170625560718
0.06993006993006994 15.504107530395347
0.06451612903225806 14.31675443223304
0.25 35.95073373078412
0.14705882352941177 21.609892469126773
0.11235955056179775 16.592538917521793
0.09345794392523364 13.830982986899008
